<a href="https://colab.research.google.com/github/OleksiiLatypov/Practical_Deep_Learning_with_PyTorch/blob/main/week3/materialist_furniture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'imaterialist-challenge-furniture-2018:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F8220%2F44216%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240509%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240509T202118Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6009eb825641852462a48c9adf726e1bcdf11f88f6ab4e5770659274b96b9461c31872f0e9bd82bb017c197423b32c2964dbd7bf15344aad20e39373137acd74ff004812059e4c2f111887af7b431287b48b4f4f80a3ae1ba929c1db5cb8da55d942fc98e3da366d46c6a8224ddba1a1c66afe2cd3adcbf9f6ee40b0b0cbdfaaa5e7d9ce0fd5da0322b3a7b693da66507cf94fcd42e9153b69f984ee855d686047b9ef84cb81b7c616ec0956fc72f63695a6a4a048d28404986057f769b26ee2ba01ec45c1d44a7b0819f28f216e289a90ba710741dfd404a30fcc18a827daa4782aed74c078992829e430f52d763411539e63de24e63ca80d7c9e5084f70c60'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imaterialist-challenge-furniture-2018/validation.json
/kaggle/input/imaterialist-challenge-furniture-2018/sample_submission_randomlabel.csv
/kaggle/input/imaterialist-challenge-furniture-2018/train.json
/kaggle/input/imaterialist-challenge-furniture-2018/test.json


In [ ]:
import os, shutil
import random
import json
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from multiprocessing.pool import ThreadPool as Pool

import matplotlib.pyplot as plt
import seaborn as sns
import urllib3

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset

import torchvision
from torchvision import transforms, models
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda


%matplotlib inline

In [ ]:
class Config:
    num_epochs = 10
    learning_rate = 0.001
    num_classes = 129
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    train_data_root = '/kaggle/input/imaterialist-challenge-furniture-2018/train.json'
    test_data_root = '/kaggle/input/imaterialist-challenge-furniture-2018/test.json'
    validation_data_root = '/kaggle/input/imaterialist-challenge-furniture-2018/validation.json'
    batch_size = 16
    num_workers = 2
    anotation_file_root = '/kaggle/input/imaterialist-furniture-fgvc5/{}.csv'
    images_path = '/kaggle/input/imaterialist-furniture-fgvc5/images/{}'
    test_anotation_file_root = '/kaggle/working/test_df.csv'

    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    resize_to = 256
    img_size = 224


In [ ]:
train_data = pd.read_json(Config.train_data_root)
test_data = pd.read_json(Config.test_data_root)
validation_data = pd.read_json(Config.validation_data_root)
print(f'Train data: {train_data.shape}')
print(f'Test data: {test_data.shape}')


Train data: (194828, 2)
Test data: (12800, 1)
